#Тест скорости инференса
В данном ноутбуке выполняется замер скорости инференса модели с рандомными весами.
Для этого скачивается проект, собирается модель, загружается тестовый набор изображений, выполняется предобработка одного из выбранных избражений и наконец производиться замер времени инференса.

In [ ]:
import sys
import os
import gdown
import numpy as np

# Устанавливаем и импортируем модуль import_ipynb. Он необходим для возможности 
# импорта файлов .ipynb 
dist_packages = ! ls /usr/local/lib/python3.7/dist-packages
if 'import_ipynb.py' not in sys.modules and 'import_ipynb.py' not in dist_packages:
    ! pip install import-ipynb
    
import import_ipynb

# Клонируем проект. Далее из проекта нам понадобиться один файл,
# файл сборки модели - Model.ipynb.
if not os.path.exists('skillbox_thesis_project'):
    ! git clone https://github.com/GrinkoL/skillbox_thesis_project.git

In [ ]:
# Загружаем и распаковываем тестовый датасет, а также удаляем загруженный архив.
# Из него мы возьмём одно изображения, на котором проверим скорость инференса.
if not os.path.exists('test'):
    if not os.path.exists('test.zip'):
        gdown.download('https://drive.google.com/uc?id=1bGHeWeWYXj5biL9s-qTc9gyv91WNAbWE', 'test.zip', False)
    ! unzip -j test.zip -d test > /dev/null
    ! rm test.zip

Downloading...
From: https://drive.google.com/uc?id=1bGHeWeWYXj5biL9s-qTc9gyv91WNAbWE
To: /content/test.zip
222MB [00:03, 72.0MB/s]


In [ ]:
# Клонируем проект. Далее нам понадобиться файл model.ipynb, лежащий в корне
# проекта.
if not os.path.exists('skillbox_thesis_project'):
    ! git clone https://github.com/GrinkoL/skillbox_thesis_project.git

In [ ]:
from skillbox_thesis_project.model import assemble_model

# Cобираем модель
model = assemble_model()

importing Jupyter notebook from model.ipynb
  Running command git clone -q https://github.com/rcmalli/keras-vggface.git /tmp/pip-req-build-fxnix9v9
580075520/580070376 [==============================] - 8s 0us/step


In [ ]:
from tensorflow.keras import backend as K

#  Определяем функцию предобработки изображения
def preprocess_input(path_img):
        img = image.load_img(path_img, target_size=(224,224))
        x_temp = image.img_to_array(img)
        data_format = K.image_data_format()
        assert data_format in {'channels_last', 'channels_first'}

        if data_format == 'channels_first':
            x_temp = x_temp[:, ::-1, ...]
            x_temp[:, 0, :, :] -= 93.5940
            x_temp[:, 1, :, :] -= 104.7624
            x_temp[:, 2, :, :] -= 129.1863
        else:
            x_temp = x_temp[..., ::-1]
            x_temp[..., 0] -= 93.5940
            x_temp[..., 1] -= 104.7624
            x_temp[..., 2] -= 129.1863

        return x_temp[None,...]

In [ ]:
from tensorflow.keras.preprocessing import image

img_path = 'test/0.jpg'
img = preprocess_input(img_path)

In [ ]:
# ВНИМАНИЕ! Первый запуск инференса с аппаратным ускорителем происходит долго,
# порядка нескольких секунд, поэтому замерять нужно со второго запуска.

%%time
np.argmax(model(img))

CPU times: user 14.4 ms, sys: 5.67 ms, total: 20.1 ms
Wall time: 21.6 ms


1

#Вывод:
Созданная нами модель имеет скорость инференса, равную ~ 22ms на одном изображении. Согласно техзаданию к дипломному проекту, скорость инференса не должна превышать 330 ms, у нас же она на порядок меньше.